In [1]:
"""
BIOCOMPUTER

Autor: Stanisław Wojtków
"""

import sys
import numpy as np
from typing import Dict, List, Optional
import gc

try:
    import libsbml
    LIBSBML_AVAILABLE = True
except ImportError:
    LIBSBML_AVAILABLE = False
    print("BLAD: libsbml nie jest zainstalowane!")

try:
    import nupack
    from nupack import *
    NUPACK_AVAILABLE = True
except ImportError:
    NUPACK_AVAILABLE = False
    print("BLAD: NUPACK nie jest zainstalowane!")



# KONFIGURACJA
COMPARTMENT_VOLUME_L = 1e-6
GATE_EXCESS_RATIO = 5.0

MATURE_SEQUENCES = {
    "miR21": "UAGCUUAUCAGACUGAUGUUGA",
    "miR155": "UUAAUGCUAAUCGUGAUAGGGGU",
    "miR34": "UGGCAGUGUCUUAGCUGGUUGU"
}

DEFAULT_PARAMS = {
    'k_on_bind21': 1e6, 'k_off_bind21': 1e-3,
    'k_on_bind155': 1e6, 'k_off_bind155': 1e-3,
    'k_and': 1e5, 'k_cat_report': 0.1,
    'k_inhibit': 1e6, 'k_uninhibit': 1e-3,
    'k_leak_gate21': 1e-5,
    'k_cross_21_155': 1e4, 'k_uncross_21_155': 1e-1,
    'k_cross_21_34': 1e4, 'k_uncross_21_34': 1e-1,
    'k_cross_155_21': 1e4, 'k_uncross_155_21': 1e-1,
    'k_cross_155_34': 1e4, 'k_uncross_155_34': 1e-1,
    'k_cross_34_21': 1e4, 'k_uncross_34_21': 1e-1,
    'k_cross_34_155': 1e4, 'k_uncross_34_155': 1e-1,
    'k_deg_Int21': 1e-3, 'k_deg_Int155': 1e-3,
    'k_deg_Output': 1e-3, 'k_deg_Signal': 1e-3,
    'k_deg_miR21': 1e-3, 'k_deg_miR155': 1e-3, 'k_deg_miR34': 1e-3,
    'k_deg_Reporter': 1e-3, 'k_deg_Reporter_Inhibited': 1e-3,
    'k_deg_Gate21': 1e-3, 'k_deg_Gate155': 1e-3, 'k_deg_Gate34': 1e-3,
}

INITIAL_CONCENTRATIONS = {
    'miR21': 1e-6, 'miR155': 1e-6, 'miR34': 0.0,
    'Gate21': 5e-6, 'Gate155': 5e-6, 'Gate34': 1e-6,
    'Reporter': 1e-6, 'Int21': 0.0, 'Int155': 0.0,
    'Output': 0.0, 'Signal': 0.0, 'Reporter_Inhibited': 0.0,
    'Crosstalk_21_155': 0.0, 'Crosstalk_21_34': 0.0,
    'Crosstalk_155_21': 0.0, 'Crosstalk_155_34': 0.0,
    'Crosstalk_34_21': 0.0, 'Crosstalk_34_155': 0.0,
}


class AdvancedNUPACKAnalyzer:
    
    def __init__(self, temperature_celsius=37, use_toeholds=True):
        if not NUPACK_AVAILABLE:
            raise ImportError("NUPACK nie jest dostepny!")
        
        self.model = Model(material='rna', celsius=temperature_celsius)
        self.R = 1.987e-3
        self.T = temperature_celsius + 273.15
        self.k_on_diffusion = 1e6
        self.use_toeholds = use_toeholds
        self.toehold_length = 8
        
        self.results = {
            'binding': {},
            'crosstalk': {},
            'structures': {},
            'gates': {},
            'folding_penalties': {},
            'design_quality': {},
            'orthogonality_score': 0.0,
        }
    
    def design_gates_with_tube_design(self, sequences_dict, num_trials=15):
        
        all_gates = {}
        all_mir_names = list(sequences_dict.keys())
        
        # Zdefiniuj domeny miRNA
        mir_domains = {}
        mir_strands = {}
        
        for mir_name, mir_seq in sequences_dict.items():
            domain = Domain(mir_seq, name=f'domain_{mir_name}')
            mir_domains[mir_name] = domain
            mir_strands[mir_name] = TargetStrand([domain], name=mir_name)
        
        # Zdefiniuj domeny bramek
        gate_domains = {}
        toehold_domains = {}
        gate_strands = {}
        
        for mir_name, mir_seq in sequences_dict.items():
            gate_name = f"Gate{mir_name.replace('miR', '')}"
            
            if self.use_toeholds and mir_name == 'miR34':
                toehold_domains[gate_name] = Domain(
                    f'N{self.toehold_length}',
                    name=f'toehold_{gate_name}'
                )
                gate_domains[gate_name] = Domain(
                    f'N{len(mir_seq)}',
                    name=f'binding_{gate_name}'
                )
                gate_strands[gate_name] = TargetStrand(
                    [toehold_domains[gate_name], gate_domains[gate_name]],
                    name=gate_name
                )
            else:
                gate_domains[gate_name] = Domain(
                    f'N{len(mir_seq)}',
                    name=f'domain_{gate_name}'
                )
                gate_strands[gate_name] = TargetStrand(
                    [gate_domains[gate_name]],
                    name=gate_name
                )
        
        # Zdefiniuj ON-TARGET complexes
        on_target_complexes = {}
        
        for mir_name, mir_seq in sequences_dict.items():
            gate_name = f"Gate{mir_name.replace('miR', '')}"
            mir_len = len(mir_seq)
            
            if self.use_toeholds and mir_name == 'miR34':
                structure = '.' * self.toehold_length + '(' * mir_len + '+' + ')' * mir_len
            else:
                structure = '(' * mir_len + '+' + ')' * mir_len
            
            complex_name = f'OnTarget_{gate_name}_{mir_name}'
            on_target_complexes[complex_name] = TargetComplex(
                [gate_strands[gate_name], mir_strands[mir_name]],
                structure,
                name=complex_name
            )
        
        # Zdefiniuj OFF-TARGET complexes
        off_target_complexes = {}
        
        print("\n  Definiuje off-targets (crosstalk pairs):")
        for mir_name in all_mir_names:
            for other_mir_name in all_mir_names:
                if mir_name == other_mir_name:
                    continue
                
                gate_name = f"Gate{other_mir_name.replace('miR', '')}"
                mir_len = len(sequences_dict[mir_name])
                gate_len = len(sequences_dict[other_mir_name])
                
                if self.use_toeholds and other_mir_name == 'miR34':
                    gate_total_len = self.toehold_length + gate_len
                else:
                    gate_total_len = gate_len
                
                structure = '.' * gate_total_len + '+' + '.' * mir_len
                
                complex_name = f'OffTarget_{mir_name}_{gate_name}'
                off_target_complexes[complex_name] = TargetComplex(
                    [gate_strands[gate_name], mir_strands[mir_name]],
                    structure,
                    name=complex_name
                )
                
                print(f"    {complex_name}: {len(structure)} nt")
        
        # Zdefiniuj TargetTube
        target_tube = TargetTube(
            on_targets={c: 1e-6 for c in on_target_complexes.values()},
            off_targets=SetSpec(max_size=2),
            name='main_tube'
        )
        
        # Zdefiniuj CONSTRAINTS
        
        hard_constraints = []
        
        for domain in gate_domains.values():
            hard_constraints.append(
                Pattern(['AAAA', 'UUUU', 'GGGG', 'CCCC'], 
                        scope=[domain])
            )
        
        for domain in gate_domains.values():
            hard_constraints.append(
                Diversity(word=4, types=2, scope=[domain])
            )
        
        soft_constraints = []
        
        for domain in gate_domains.values():
            soft_constraints.append(
                Pattern(['AAAAA', 'UUUUU', 'GGGGG', 'CCCCC'], 
                        scope=[domain], weight=5.0)
            )
        
        # GC content (40-60%)
        for gate_name, strand in gate_strands.items():
            if self.use_toeholds and gate_name == 'Gate34':
                domain = gate_domains[gate_name]
                ref_len = len(sequences_dict['miR34'])
            else:
                domain = gate_domains[gate_name]
                ref_len = len(sequences_dict[gate_name.replace('Gate', 'miR')])
            
            ref_gc50 = 'G' * (ref_len // 2) + 'C' * (ref_len - ref_len // 2)
            soft_constraints.append(
                Similarity([domain], ref_gc50, limits=[0.4, 0.6], weight=1.0)
            )
        
        
        # SSM dla krótkich słów (word=4) - wysoka waga
        ssm_short = SSM(
            word=4,
            scope=list(on_target_complexes.values()),
            weight=15.0
        )
        soft_constraints.append(ssm_short)
        
        # SSM dla średnich słów (word=6) - średnia waga
        ssm_medium = SSM(
            word=6,
            scope=list(on_target_complexes.values()),
            weight=10.0
        )
        soft_constraints.append(ssm_medium)
        
        # SSM dla długich słów (word=8) - niska waga
        ssm_long = SSM(
            word=8,
            scope=list(on_target_complexes.values()),
            weight=5.0
        )
        soft_constraints.append(ssm_long)
        
        # Uruchom DESIGN
        print(f"\n  Uruchamiam tube_design (trials={num_trials})...")
        
        try:
            design_job = tube_design(
                tubes=[target_tube],
                hard_constraints=hard_constraints,
                soft_constraints=soft_constraints,
                model=self.model,
                options=DesignOptions(
                    wobble_mutations=False,
                    seed=42
                )
            )
            
            design_results = design_job.run(trials=num_trials)
            
            print(f"  Otrzymano {len(design_results)} wyników")
            
            # Wybierz najlepszy wynik
            best_result = None
            best_defect = float('inf')
            
            for idx, result in enumerate(design_results):
                try:
                    if hasattr(result, 'defects'):
                        try:
                            defect_val = float(str(result.defects).split('=')[1].split()[0])
                            print(f" Trial {idx}: ensemble_defect = {defect_val:.6f}")
                            
                            if defect_val < best_defect:
                                best_defect = defect_val
                                best_result = result
                        except:
                            if best_result is None:
                                best_result = result
                                best_defect = 0.0
                    else:
                        if best_result is None:
                            best_result = result
                            best_defect = 0.0
                            
                except Exception as e:
                    print(f" WARNING Trial {idx}: {e}")
                    continue
            
            if best_result is None:
                raise ValueError("Nie znaleziono poprawnego wyniku designu")
            
            print(f" Najlepszy wynik: ensemble_defect = {best_defect:.6f}")
            self.results['design_quality']['best_defect'] = best_defect
            self.results['design_quality']['num_trials'] = len(design_results)
            
            # Ekstrakcja sekwencji
            for gate_name, strand in gate_strands.items():
                try:
                    designed_seq = str(best_result.to_analysis(strand))
                    all_gates[gate_name] = designed_seq
                except Exception as e:
                    print(f" Blad ekstrakcji {gate_name}: {e}")
                    all_gates[gate_name] = self._simple_complement(
                        sequences_dict[gate_name.replace('Gate', 'miR')]
                    )
            
        except Exception as e:
            print(f" Design nie powiodl sie: {e}")
            print(f" Uzywam 'fallback'")
            import traceback
            traceback.print_exc()
            
            # Fallback
            for mir_name, mir_seq in sequences_dict.items():
                gate_name = f"Gate{mir_name.replace('miR', '')}"
                all_gates[gate_name] = self._simple_complement(mir_seq)
        
        self.results['gates'] = all_gates
        
        print("\n  Zaprojektowane bramki:")
        for gate_name, gate_seq in all_gates.items():
            print(f"    {gate_name:8s}: {gate_seq}")
        
        return all_gates
    
    def _simple_complement(self, seq):
        comp = {'A': 'U', 'U': 'A', 'G': 'C', 'C': 'G'}
        return ''.join(comp[b] for b in reversed(seq))
    
    def calculate_folding_penalty(self, dG_fold):
        if dG_fold >= 0:
            return 1.0
        return np.exp(dG_fold / (self.R * self.T))
    
    def analyze_secondary_structure(self, seq, name):
        try:
            strand = Strand(seq, name=name)
            result = mfe(strands=[strand], model=self.model)
            
            structure_info = {
                'sequence': seq,
                'structure': str(result[0].structure),
                'dG_fold': result[0].energy,
                'is_folded': result[0].energy < -2.0
            }
            
            penalty = self.calculate_folding_penalty(structure_info['dG_fold'])
            self.results['folding_penalties'][name] = penalty
            self.results['structures'][name] = structure_info
            
            del strand, result
            gc.collect()
            
            return structure_info
            
        except Exception as e:
            print(f"  WARNING Blad analizy {name}: {e}")
            return {
                'sequence': seq,
                'structure': '.' * len(seq),
                'dG_fold': 0.0,
                'is_folded': False
            }
    
    def analyze_binding(self, target_seq, gate_seq, target_name, gate_name):
        try:
            strand_target = Strand(target_seq, name='target')
            strand_gate = Strand(gate_seq, name='gate')
            
            tube = Tube(
                strands={strand_target: 1e-6, strand_gate: 1e-6},
                complexes=SetSpec(max_size=2),
                name='binding_tube'
            )
            
            tube_result = tube_analysis(
                tubes=[tube],
                model=self.model,
                compute=['pfunc']
            )
            
            complex_tg = Complex([strand_target, strand_gate])
            complex_t = Complex([strand_target])
            complex_g = Complex([strand_gate])
            
            dG_complex = tube_result[complex_tg].free_energy
            dG_target = tube_result[complex_t].free_energy
            dG_gate = tube_result[complex_g].free_energy
            
            dG_bind = dG_complex - dG_target - dG_gate
            
            Kd = np.exp(dG_bind / (self.R * self.T))
            k_off = self.k_on_diffusion * Kd
            
            k_on_corrected = self.k_on_diffusion
            if target_name in self.results['folding_penalties']:
                penalty = self.results['folding_penalties'][target_name]
                k_on_corrected *= penalty
            
            result = {
                'target': target_name,
                'gate': gate_name,
                'dG_bind': dG_bind,
                'K_d_M': Kd,
                'K_d_nM': Kd * 1e9,
                'k_on_base': self.k_on_diffusion,
                'k_on_corrected': k_on_corrected,
                'k_off': k_off,
                'folding_penalty': self.results['folding_penalties'].get(target_name, 1.0)
            }
            
            del strand_target, strand_gate, tube, tube_result
            gc.collect()
            
            return result
            
        except Exception as e:
            print(f" Blad analizy {target_name}+{gate_name}: {e}")
            return {
                'target': target_name,
                'gate': gate_name,
                'dG_bind': 5.0,
                'K_d_M': 1e-3,
                'K_d_nM': 1e6,
                'k_on_base': self.k_on_diffusion,
                'k_on_corrected': self.k_on_diffusion,
                'k_off': 1e3,
                'folding_penalty': 1.0
            }
    
    def analyze_all_structures(self, sequences_dict):
        print("\n[ANALIZA] Struktury drugorzędowe + folding penalties...")
        
        for name, seq in sequences_dict.items():
            info = self.analyze_secondary_structure(seq, name)
            penalty = self.results['folding_penalties'].get(name, 1.0)
            
            status = "WARNING FOLDED" if info['is_folded'] else "OK"
            print(f"  {name:8s}: dG = {info['dG_fold']:6.2f} kcal/mol, "
                  f"penalty = {penalty:.4f}  [{status}]")
    
    def analyze_all_binding(self, sequences_dict):
        print("\n[ANALIZA] Wiązanie on-target...")
        
        for mir_name, mir_seq in sequences_dict.items():
            gate_name = f"Gate{mir_name.replace('miR', '')}"
            gate_seq = self.results['gates'][gate_name]
            
            result = self.analyze_binding(mir_seq, gate_seq, mir_name, gate_name)
            self.results['binding'][f"{mir_name}+{gate_name}"] = result
            
            print(f"  {mir_name:8s} + {gate_name:9s}:")
            print(f"    K_d = {result['K_d_nM']:8.2f} nM")
            print(f"    k_on = {result['k_on_corrected']:.2e} M^-1 s^-1 "
                  f"(penalty: {result['folding_penalty']:.4f})")
    
    def analyze_crosstalk(self, sequences_dict):
        print("\n[ANALIZA] Crosstalk...")
        
        mir_names = list(sequences_dict.keys())
        gate_names = [f"Gate{name.replace('miR', '')}" for name in mir_names]
        
        crosstalk_strong = 0
        crosstalk_moderate = 0
        crosstalk_weak = 0
        
        for mir_name in mir_names:
            mir_seq = sequences_dict[mir_name]
            for gate_name in gate_names:
                if gate_name == f"Gate{mir_name.replace('miR', '')}":
                    continue
                
                gate_seq = self.results['gates'][gate_name]
                result = self.analyze_binding(mir_seq, gate_seq, mir_name, gate_name)
                
                pair_name = f"{mir_name}+{gate_name}"
                self.results['crosstalk'][pair_name] = result
                
                K_d_nM = result['K_d_nM']
                if K_d_nM < 100:
                    level = "STRONG"
                    crosstalk_strong += 1
                elif K_d_nM < 1000:
                    level = "MODERATE"
                    crosstalk_moderate += 1
                else:
                    level = "WEAK"
                    crosstalk_weak += 1
                
                print(f"  {pair_name:20s}: K_d = {K_d_nM:7.2f} nM  [{level}]")
        
        total_pairs = crosstalk_strong + crosstalk_moderate + crosstalk_weak
        if total_pairs > 0:
            good_pairs = crosstalk_weak + crosstalk_moderate
            self.results['orthogonality_score'] = 100.0 * good_pairs / total_pairs
        
        print(f"\n  Statystyki:")
        print(f"    Silny:       {crosstalk_strong}/{total_pairs}")
        print(f"    Umiarkowany: {crosstalk_moderate}/{total_pairs}")
        print(f"    Slaby:       {crosstalk_weak}/{total_pairs}")
        print(f"    Orthogonality: {self.results['orthogonality_score']:.1f}%")
    
    def generate_sbml_params(self):
        params = DEFAULT_PARAMS.copy()
        
        for pair_name, result in self.results['binding'].items():
            mir_num = result['target'].replace('miR', '')
            params[f'k_on_bind{mir_num}'] = result['k_on_corrected']
            params[f'k_off_bind{mir_num}'] = result['k_off']
        
        crosstalk_map = {
            'miR21+Gate155': ('k_cross_21_155', 'k_uncross_21_155'),
            'miR21+Gate34': ('k_cross_21_34', 'k_uncross_21_34'),
            'miR155+Gate21': ('k_cross_155_21', 'k_uncross_155_21'),
            'miR155+Gate34': ('k_cross_155_34', 'k_uncross_155_34'),
            'miR34+Gate21': ('k_cross_34_21', 'k_uncross_34_21'),
            'miR34+Gate155': ('k_cross_34_155', 'k_uncross_34_155'),
        }
        
        for pair_name, (k_on_name, k_off_name) in crosstalk_map.items():
            if pair_name in self.results['crosstalk']:
                result = self.results['crosstalk'][pair_name]
                params[k_on_name] = result['k_on_corrected']
                params[k_off_name] = result['k_off']
        
        return params
    
    def print_summary(self):
        print("PODSUMOWANIE")
        
        if 'best_defect' in self.results['design_quality']:
            print(f"\n1. DESIGN QUALITY:")
            print(f"   Ensemble defect: {self.results['design_quality']['best_defect']:.6f}")
            print(f"   (nizsze = lepsze, cel: < 0.1)")
        
        print(f"\n2. FOLDING PENALTIES:")
        for name, penalty in self.results['folding_penalties'].items():
            k_on = self.k_on_diffusion * penalty
            status = "WARNING" if penalty < 0.1 else "OK"
            print(f"   {name:8s}: {penalty:.4f} [{status}] (k_on: {k_on:.2e} M^-1 s^-1)")
        
        print(f"\n3. ORTHOGONALITY: {self.results['orthogonality_score']:.1f}%")
        
        print("\n" + "="*70)



# SBML
def create_sbml_model(params):
    if not LIBSBML_AVAILABLE:
        raise ImportError("python-libsbml wymagane!")
    
    doc = libsbml.SBMLDocument(3, 1)
    model = doc.createModel()
    model.setId("biocomputer_tube_design")
    model.setName("Biocomputer with NUPACK tube_design")
    
    compartment = model.createCompartment()
    compartment.setId("c1")
    compartment.setConstant(True)
    compartment.setSize(COMPARTMENT_VOLUME_L)
    compartment.setUnits("litre")
    
    for species_id, init_conc in INITIAL_CONCENTRATIONS.items():
        species = model.createSpecies()
        species.setId(species_id)
        species.setCompartment("c1")
        species.setInitialConcentration(init_conc)
        species.setHasOnlySubstanceUnits(False)
        species.setConstant(False)
        species.setBoundaryCondition(False)
    
    for param_id, param_value in params.items():
        parameter = model.createParameter()
        parameter.setId(param_id)
        parameter.setValue(param_value)
        parameter.setConstant(False)
    
    add_reactions(model)
    return doc


def add_reactions(model):
    """Dodaje reakcje do modelu SBML"""
    
    # ON-TARGET BINDING
    add_reaction(model, "bind21", 
                 [("Gate21", 1), ("miR21", 1)], [("Int21", 1)],
                 "k_on_bind21 * Gate21 * miR21")
    add_reaction(model, "unbind21",
                 [("Int21", 1)], [("Gate21", 1), ("miR21", 1)],
                 "k_off_bind21 * Int21")
    
    add_reaction(model, "bind155",
                 [("Gate155", 1), ("miR155", 1)], [("Int155", 1)],
                 "k_on_bind155 * Gate155 * miR155")
    add_reaction(model, "unbind155",
                 [("Int155", 1)], [("Gate155", 1), ("miR155", 1)],
                 "k_off_bind155 * Int155")
    
    # CROSSTALK
    crosstalk_pairs = [
        ("miR21", "Gate155", "Crosstalk_21_155", "k_cross_21_155", "k_uncross_21_155"),
        ("miR21", "Gate34", "Crosstalk_21_34", "k_cross_21_34", "k_uncross_21_34"),
        ("miR155", "Gate21", "Crosstalk_155_21", "k_cross_155_21", "k_uncross_155_21"),
        ("miR155", "Gate34", "Crosstalk_155_34", "k_cross_155_34", "k_uncross_155_34"),
        ("miR34", "Gate21", "Crosstalk_34_21", "k_cross_34_21", "k_uncross_34_21"),
        ("miR34", "Gate155", "Crosstalk_34_155", "k_cross_34_155", "k_uncross_34_155"),
    ]
    
    for mir, gate, product, k_on, k_off in crosstalk_pairs:
        add_reaction(model, f"cross_bind_{product}",
                     [(mir, 1), (gate, 1)], [(product, 1)],
                     f"{k_on} * {mir} * {gate}")
        add_reaction(model, f"cross_unbind_{product}",
                     [(product, 1)], [(mir, 1), (gate, 1)],
                     f"{k_off} * {product}")
    
    # AND gate
    add_reaction(model, "and_generate_output",
                 [("Int21", 1), ("Int155", 1)], [("Output", 1)],
                 "k_and * Int21 * Int155")
    
    # Reporter activation
    add_reaction(model, "reporter_activate",
                 [("Output", 1), ("Reporter", 1)], [("Signal", 1)],
                 "k_cat_report * Output * Reporter")
    
    # NOT gate
    add_reaction(model, "inhibit_reporter",
                 [("miR34", 1), ("Reporter", 1)], [("Reporter_Inhibited", 1)],
                 "k_inhibit * miR34 * Reporter")
    add_reaction(model, "uninhibit_reporter",
                 [("Reporter_Inhibited", 1)], [("miR34", 1), ("Reporter", 1)],
                 "k_uninhibit * Reporter_Inhibited")
    
    # Leakage
    add_reaction(model, "leak_gate21",
                 [("Gate21", 1)], [("Output", 1)],
                 "k_leak_gate21 * Gate21")
    
    # Degradations
    degradable = ["Int21", "Int155", "Output", "Signal",
                  "miR21", "miR155", "miR34",
                  "Reporter", "Reporter_Inhibited",
                  "Gate21", "Gate155", "Gate34",
                  "Crosstalk_21_155", "Crosstalk_21_34",
                  "Crosstalk_155_21", "Crosstalk_155_34",
                  "Crosstalk_34_21", "Crosstalk_34_155"]
    
    for species_id in degradable:
        k_deg = f"k_deg_{species_id}" if f"k_deg_{species_id}" in DEFAULT_PARAMS else "k_deg_Int21"
        add_reaction(model, f"deg_{species_id}",
                     [(species_id, 1)], [],
                     f"{k_deg} * {species_id}")


def add_reaction(model, reaction_id, reactants, products, formula):
    """Pomocnicza funkcja dodająca reakcję"""
    reaction = model.createReaction()
    reaction.setId(reaction_id)
    reaction.setReversible(False)
    reaction.setFast(False)
    
    for species_id, stoich in reactants:
        species_ref = reaction.createReactant()
        species_ref.setSpecies(species_id)
        species_ref.setStoichiometry(stoich)
        species_ref.setConstant(True)
    
    for species_id, stoich in products:
        species_ref = reaction.createProduct()
        species_ref.setSpecies(species_id)
        species_ref.setStoichiometry(stoich)
        species_ref.setConstant(True)
    
    kinetic_law = reaction.createKineticLaw()
    math_ast = libsbml.parseL3Formula(formula)
    kinetic_law.setMath(math_ast)


def export_sbml(doc, filename):
    """Eksportuje SBML"""
    writer = libsbml.SBMLWriter()
    success = writer.writeSBMLToFile(doc, filename)
    
    if success:
        print(f"\nPlik SBML wyeksportowany: {filename}")
        return True
    else:
        print(f"\nERROR Blad zapisu SBML: {filename}")
        return False


# MAIN
def main():    
    print(f"\nLogika: Signal = (miR21 AND miR155) AND NOT miR34")
    
    if not LIBSBML_AVAILABLE or not NUPACK_AVAILABLE:
        print("\nERROR: Brakuje wymaganych bibliotek!")
        return 1
    
    try:
        # Inicjalizuj analyzer
        analyzer = AdvancedNUPACKAnalyzer(
            temperature_celsius=37,
            use_toeholds=True
        )
        
        
        analyzer.design_gates_with_tube_design(
            MATURE_SEQUENCES,
            num_trials=50
        )
        
        # Analiza
        print("\n" + " ANALIZA TERMODYNAMICZNA")
        
        analyzer.analyze_all_structures(MATURE_SEQUENCES)
        analyzer.analyze_all_binding(MATURE_SEQUENCES)
        analyzer.analyze_crosstalk(MATURE_SEQUENCES)
        
        # Podsumowanie
        print("\n")
        analyzer.print_summary()
        
        # Generuj SBML
        print("\nGENEROWANIE SBML")

        
        params = analyzer.generate_sbml_params()
        doc = create_sbml_model(params)
        
        output_file = "biocomputer_tube_design.xml"
        success = export_sbml(doc, output_file)
        
        if success:
            print(f"\nPlik: {output_file}")
            
            return 0
        else:
            return 1
    
    except Exception as e:
        print(f"\nERROR: {e}")
        import traceback
        traceback.print_exc()
        return 1


if __name__ == "__main__":
    exit(main())


Logika: Signal = (miR21 AND miR155) AND NOT miR34

  Definiuje off-targets (crosstalk pairs):
    OffTarget_miR21_Gate155: 46 nt
    OffTarget_miR21_Gate34: 53 nt
    OffTarget_miR155_Gate21: 46 nt
    OffTarget_miR155_Gate34: 54 nt
    OffTarget_miR34_Gate21: 45 nt
    OffTarget_miR34_Gate155: 46 nt

  Uruchamiam tube_design (trials=50)...
  Otrzymano 50 wyników
 Najlepszy wynik: ensemble_defect = 0.000000

  Zaprojektowane bramki:
    Gate21  : UCAGCGUUAGUUUGGUAAGCUA
    Gate155 : GCCUCUGUCACGAUUAGCAUUGA
    Gate34  : GGAAUACGGCGGUCGGCUGGGACACUGCCA

 ANALIZA TERMODYNAMICZNA

[ANALIZA] Struktury drugorzędowe + folding penalties...
  miR21   : dG =  -1.10 kcal/mol, penalty = 0.1681  [OK]
  miR155  : dG =   0.00 kcal/mol, penalty = 1.0000  [OK]
  miR34   : dG =  -4.28 kcal/mol, penalty = 0.0010  [WARNING FOLDED]

[ANALIZA] Wiązanie on-target...
  miR21    + Gate21   :
    K_d =     0.00 nM
    k_on = 1.68e+05 M^-1 s^-1 (penalty: 0.1681)
  miR155   + Gate155  :
    K_d =     0.00 nM
   